# Enable Model Monitoring
Model Monitoring needs to be enabled in each model in order to see tracking results.

To enable Model Monitoring in any model, make sure that the `serving_fn.set_tracking()` function is included in the Model Server.

To utilize Drift measurement, make sure you supply the train set when logging the model in the
training step.

## Model Monitoring Demo
The following is a demo that you can use to test and explore the Model Monitoring feature.

In [ ]:
# Set project name
project = ""

### Deploy Model Servers

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris

from mlrun import import_function, get_dataitem
from mlrun import projects
from mlrun.platforms import auto_mount

proj = projects.new_project(project)

get_dataitem("https://s3.wasabisys.com/iguazio/models/iris/model.pkl").download(
    "model.pkl")

iris = load_iris()
train_set = pd.DataFrame(iris['data'],
                         columns=['sepal_length_cm', 'sepal_width_cm',
                                  'petal_length_cm', 'petal_width_cm'])

model_names = [
    "sklearn_ensemble_RandomForestClassifier",
    "sklearn_linear_model_LogisticRegression",
    "sklearn_ensemble_AdaBoostClassifier"
]

serving_fn = import_function('hub://v2_model_server').apply(auto_mount())

for name in model_names:
    proj.log_model(name, model_file="model.pkl", training_set=train_set)
    serving_fn.add_model(name,
                         model_path=f"store://models/{project}/{name}:latest")

serving_fn.metadata.project = project
serving_fn.set_tracking()
serving_fn.deploy()

### Deploy Stream Processing

In [ ]:
import os

from mlrun import import_function
from mlrun.platforms import mount_v3io
from mlrun.runtimes import RemoteRuntime
import json

fn: RemoteRuntime = import_function("hub://model_monitoring_stream")

fn.add_v3io_stream_trigger(
    stream_path=f"projects/{project}/model-endpoints/stream",
    name="monitoring_stream_trigger",
)

fn.set_env("MODEL_MONITORING_PARAMETERS", json.dumps(
    {"project": project, "v3io_framesd": os.environ.get("V3IO_FRAMESD")}))

fn.metadata.project = project
fn.apply(mount_v3io())
fn.deploy()

### Deploy Batch Processing

In [ ]:
from mlrun import import_function
from mlrun.platforms import mount_v3io
from mlrun.runtimes import KubejobRuntime

fn: KubejobRuntime = import_function("hub://model_monitoring_batch")
fn.metadata.project = project
fn.apply(mount_v3io())
fn.run(name='model-monitoring-batch', schedule="0 */1 * * *",
       params={"project": project})

### Simulating Requests

In [ ]:
import json
from time import sleep
from random import choice, uniform
from sklearn.datasets import load_iris

iris = load_iris()
iris_data = iris['data'].tolist()

while True:
    for name in model_names:
        data_point = choice(iris_data)
        serving_fn.invoke(f'v2/models/{name}/infer',
                          json.dumps({'inputs': [data_point]}))
        sleep(uniform(0.1, 0.4))
    sleep(uniform(0.2, 1.7))
